In [ ]:
%pip install langchain openai langchain-openai langchain-community langchainhub langchain_ollama

In [12]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

In [13]:
from dotenv import load_dotenv
import os

In [ ]:
%load_ext dotenv
%dotenv

In [15]:
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [16]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
llm.invoke("who are you?")

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage

In [19]:
messages = [
    SystemMessage("You are an AI assistant design to tell funny jokes. Do not answer any question that is not a joke."),
    HumanMessage("Tell me a joke."),
]

In [ ]:
response = llm.invoke(messages)
print(response.content)

In [ ]:
username = "Prince"
txt = f"Hello {username}"

print(txt)

In [22]:
from langchain_core.prompts import PromptTemplate

In [23]:
prompt_template = PromptTemplate.from_template(
    "Tell me historical fact about the {event} in {location}.",
)

In [ ]:
prompt_template.format(event="World War II", location="Europe")

In [25]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

In [26]:
system_message_str = """You are a helpful AI assistant. Given the 
following context, answer the question. If the answer can not be 
found in the context, simply say you DO NOT KNOW.
Context: {context}
"""

In [27]:
system_message_promt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=system_message_str,
    )
)

In [28]:
human_message_str = "Can you provide details on:  {question}"

In [29]:
human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template=human_message_str,
    )
)

In [30]:
messages = [system_message_promt, human_message_prompt]

In [31]:
chatbot_prompt_template = ChatPromptTemplate(
    messages=messages,
    input_variables=["context", "question"],
)


In [32]:
question = "What is the capital of India?"
context = "India is a country in Asia. It's capital is Delhi."

In [ ]:
chatbot_prompt_template.format(context=context, question=question)

In [34]:
response = llm.invoke(
    chatbot_prompt_template.format(context=context, question="What is the capital of Kenya?")
)

In [ ]:
response

In [ ]:
response.content

In [37]:
from langchain_core.output_parsers import StrOutputParser

In [38]:
chain = chatbot_prompt_template | llm | StrOutputParser()

In [ ]:
chain.invoke({
    "context": context,
    "question": question
})

In [ ]:
chain.invoke({
    "context": context,
    "question": "What is the capital of India?"
})

In [41]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent
)
from langchain.tools import tool
from langchain_core.tools import ToolException
from typing import Literal

In [42]:
operator_type = Literal["add", "multiply", "substraction", "division"]

In [43]:
@tool("Calculator-tool", return_direct=False)
def calculator(operator: operator_type, a: float, b: float) -> float:
    """A simple calculator tool that can add, multiply, substract, or divide two numbers."""
    if operator == "add":
        return a + b
    elif operator == "multiply":
        return a * b
    elif operator == "substraction":
        return a - b
    elif operator == "division":
        return a / b
    else:
        raise ToolException("Invalid operator")

In [ ]:
print(calculator.name)
print(calculator.description)
print(calculator.return_direct)
print(calculator.args)

In [ ]:
calculator.run({"operator": "add", "a": 2, "b": 3})

In [46]:
tools = [calculator]

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

In [48]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [49]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({
    "input": "What is the product of 2 and 3?",
})

In [51]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.prompts import (
    PromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate
)

In [52]:
neo4j_graph_vector_index = Neo4jVector.from_existing_graph(
    embedding = OllamaEmbeddings(
    model="llama3.1",
),
    index_name="product",
    node_label="Product",
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    text_node_properties=[
        "images",
        "gender",
        "price",
        "name",
        "mpn"
        "currency",
        "sku",
        "in_stock"
    ],
    embedding_node_property="embedding"
)

In [53]:
result = neo4j_graph_vector_index.similarity_search("Parx" , k=2)

In [ ]:
result

In [ ]:
print(result[0].metadata)

In [ ]:
result = neo4j_graph_vector_index.similarity_search(
    "Product detail", filter={"brand": "DYNK"}
)

In [ ]:
result

In [58]:
for product in result:
    clearned_page_content = dict(line.split(": ", 1) for line in product.page_content.strip().split("\n"))
    
    print(f"""
          Name: {clearned_page_content.get('name')}
          Price: {clearned_page_content.get('price')}
          Gender: {clearned_page_content.get('gender')}
          Images: {clearned_page_content.get('images')}
          In_stock: {product.metadata.get('in_stock')}
          """)

In [59]:
product_details_chat_template_str = """
Your job is to use the provided product data to answer 
questions about the description, brand, price and quality
to the E-commerce customer. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. If you don't know an answer, say you don't know.

Context: {context}
"""

In [60]:
product_details_chat_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=product_details_chat_template_str
    )
)

In [61]:
human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="Can you provide details on: {question}?"
    )
)

In [62]:
messages = [product_details_chat_system_prompt, human_prompt]

In [63]:
qa_prompt = ChatPromptTemplate(
    messages=messages,
    input_variables=["context", "question"]
)

In [64]:
llm = OllamaFunctions(model="llama3.1")

In [65]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=neo4j_graph_vector_index.as_retriever(),
    chain_type="stuff",
)

In [66]:

qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt

In [67]:
response = qa_chain.invoke("Give me details of DYNK products from database.")

In [ ]:
print(response.get("result"))

In [69]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

In [ ]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URL"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

In [71]:
graph.refresh_schema()

In [72]:
cypher_generation_template = """
Task:
Generate Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note:
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than
for you to construct a Cypher statement. Do not include any text except
the generated Cypher statement. Make sure the direction of the relationship is
correct in your queries. Make sure you alias both entities and relationships
properly. Do not run any queries that would add to or delete from
the database. Make sure to alias all statements that follow as with
statement (e.g. WITH c as customer, o.orderID as order_id).
If you need to divide numbers, make sure to
filter the denominator to be non-zero.

Examples:
# Retrieve the total number of orders placed by each customer.
MATCH (c:Customer)-[o:ORDERED_BY]->(order:Order)
RETURN c.customerID AS customer_id, COUNT(o) AS total_orders
# List the top 5 products with the highest unit price.
MATCH (p:Product)
RETURN p.productName AS product_name, p.price AS product_price
ORDER BY product_price DESC
LIMIT 5
# Find all products which is sold most.
MATCH (b:Brand)-[r:Maker]->(p:Product)
RETURN p.name AS s_no, COUNT(o) AS sold_most
String category values:
Use existing strings and values from the schema provided. 

The question is:
{question}
"""

In [73]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template=cypher_generation_template
)

In [74]:
qa_generation_template_str = """
You are an assistant that takes the results from a Neo4j Cypher query and forms a human-readable response. The query results section contains the results of a Cypher query that was generated based on a user's natural language question. The provided information is authoritative; you must never question it or use your internal knowledge to alter it. Make the answer sound like a response to the question.

Query Results:
{context}
Question:
{question}
If the provided information is empty, respond by stating that you don't know the answer. Empty information is indicated by: []
If the information is not empty, you must provide an answer using the results. If the question involves a time duration, assume the query results are in units of days unless specified otherwise.
When names are provided in the query results, such as hospital names, be cautious of any names containing commas or other punctuation. For example, 'Jones, Brown and Murray' is a single hospital name, not multiple hospitals. Ensure that any list of names is presented clearly to avoid ambiguity and make the full names easily identifiable.
Never state that you lack sufficient information if data is present in the query results. Always utilize the data provided.
Helpful Answer:
"""

In [75]:
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template_str
)

In [76]:
cypher_chain = GraphCypherQAChain.from_llm(
    allow_dangerous_requests=True,
    top_k=10,
    graph=graph,
    verbose=True,
    validate_cypher=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    qa_llm=OllamaFunctions(model="llama3.1", temperature=0),
    cypher_llm=OllamaFunctions(model="llama3.1", temperature=0),
)

In [ ]:
question = "What is the most expensive product?"
response = cypher_chain.invoke(question)

In [ ]:
print(response.get("result"))

In [ ]:
question = "List the top 5 products with the lowest price."
response = cypher_chain.invoke(question)

In [ ]:
print(response.get("result"))

In [ ]:
question = "Give me details on Parx product price , description and images."
response = cypher_chain.invoke(question)

In [ ]:
print(response.get("result"))

In [83]:
product_details_chat_template_str = """
Your job is to use the provided product data to answer 
questions about the description, brand, price and quality
to the E-commerce customer. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. 

Context: {context}
"""

In [84]:
@tool("product-qa-tool", return_direct=False)
def product_qa_tool(query: str) -> str:
    """Useful for answering questions about products made by the brands."""
    
    product_details_chat_system_prompt = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context"],
            template=product_details_chat_template_str
        )
    )
    
    human_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["question"],
            template="Can you provide details on: {question}?"
        )
    )
    
    messages = [product_details_chat_system_prompt, human_prompt]
    
    qa_prompt = ChatPromptTemplate(
        messages=messages,
        input_variables=["context", "question"]
    )
    
    llm = OllamaFunctions(model="llama3.1")

    
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=neo4j_graph_vector_index.as_retriever(),
        # ['stuff', 'map_reduce', 'refine', 'map_rerank']
        chain_type="stuff",
    )
    
    qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt
    
    response = qa_chain.invoke(query)
    
    return response.get("result")

In [ ]:
product_qa_tool.invoke("What is the price of Slim Fit Printed Casual Shirt?")

In [86]:
@tool("general-qa-tool", return_direct=False)
def general_qa_tool(query: str) -> str:
    """Useful for answering general questions about product, price, in_stock, gender, and currency."""
    response = cypher_chain.invoke(query)
    
    return response.get("result")

In [ ]:
general_qa_tool.invoke("What is the most expensive product?")

In [88]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent
)

In [ ]:
agent_prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
agent_prompt.pretty_print()

In [91]:
tools = [product_qa_tool, general_qa_tool]

In [92]:
llm = OllamaFunctions(model="llama3.1")

In [93]:
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({
    "input": "What is the most expensive product?"
})

In [ ]:
agent_executor.invoke({
    "input": "What is the price DYNK Shirt"
})

In [96]:
product_details_chat_template_str = """
Your job is to use the provided product data to answer 
questions about the description, brand, price and quality
to the E-commerce customer. Use the following context to answer questions. 
Be as detailed as possible, but don't make up any information that's 
not from the context. If you don't know an answer, say you don't know.
Context: {context}
"""

In [97]:
@tool("product-qa-tool", return_direct=True)
def product_qa_tool(query: str) -> str:
    """Useful for answering questions about products made by the company brands."""
    product_details_chat_system_prompt = SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["context"],
            template=product_details_chat_template_str
        )
    )
    
    human_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["question"],
            template="Can you provide details on: {question}?"
        )
    )
    
    messages = [product_details_chat_system_prompt, human_prompt]
    
    qa_prompt = ChatPromptTemplate(
        messages=messages,
        input_variables=["context", "question"]
    )
    
    llm = OllamaFunctions(model="llama3.1")
    
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=neo4j_graph_vector_index.as_retriever(),
        # ['stuff', 'map_reduce', 'refine', 'map_rerank']
        chain_type="stuff",
    )
    
    qa_chain.combine_documents_chain.llm_chain.prompt = qa_prompt
    
    response = qa_chain.invoke(query)
    
    return response.get("result")

In [ ]:
product_qa_tool.invoke("What are the products from Parx brand?")

In [99]:
@tool("general-qa-tool", return_direct=True)
def general_qa_tool(query: str) -> str:
    """Useful for answering general questions about product name, price, brand, availability, description and link."""
    response = cypher_chain.invoke(query)
    
    return response.get("result")

In [ ]:
general_qa_tool.invoke("What is the most made product from branded company in the database?")

In [101]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_tool_calling_agent
)

In [ ]:
agent_prompt = hub.pull("hwchase17/openai-functions-agent")


In [103]:
tools = [product_qa_tool, general_qa_tool]

In [104]:
llm = OllamaFunctions(model="llama3.1",format="json")

In [105]:
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
agent_executor.invoke(
    {
        "input": "What is the price of DYNK Shirt?",
    }
)

In [107]:
#%pip install gpt4all 